# Fetch experiments data from Neptune using [Query API](https://docs.neptune.ai/python-api/query-api.html)

This notebooks show example usage of the query API. It is set of Python methods that let you fetch experiments data from neptune. This notebook presents some use cases of analysis with the data downloaded from Neptune system.

## Methods
This notebook covers most common methods like:

1. [get_experiments()](https://docs.neptune.ai/neptune-client/docs/project.html#neptune.projects.Project.get_experiments) - get a list of the [Experiment objects](https://docs.neptune.ai/neptune-client/docs/experiment.html). We will need them to fetch data from selected experiments.
1. [get_leaderboard()](https://docs.neptune.ai/neptune-client/docs/project.html#neptune.projects.Project.get_leaderboard) - get experiments table as a pandas DataFrame. Example experiment table is [here](https://ui.neptune.ai/o/USERNAME/org/example-project/experiments?viewId=6013ecbc-416d-4e5c-973e-871e5e9010e9).
1. [get_hardware_utilization()](https://docs.neptune.ai/neptune-client/docs/experiment.html#neptune.experiments.Experiment.get_hardware_utilization) - for the Experiment in question, get hardware utilization metrics as pandas DataFrame ([example metrics](https://ui.neptune.ai/o/USERNAME/org/example-project/e/HELLO-177/monitoring)).
1. [get_logs()](https://docs.neptune.ai/neptune-client/docs/experiment.html#neptune.experiments.Experiment.get_logs) - get dict, where keys are log names and values are Channel objects.
1. [get_numeric_channels_values()](https://docs.neptune.ai/neptune-client/docs/experiment.html#neptune.experiments.Experiment.get_numeric_channels_values) - get values of numeric logs as pandas DataFrame ([example logs](https://ui.neptune.ai/o/USERNAME/org/example-project/e/HELLO-177/charts)).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from utils.tokens import NEPTUNE_API_TOKEN
import neptune
from scipy.stats import hmean
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from training_utils import problem_kind
sns.set(style="darkgrid")

# Set project to work with (as usual)

In [3]:
project = neptune.init('createrandom/mus-RQ1',
                       api_token=NEPTUNE_API_TOKEN)

# Visualize metrics

`get_experiments()` below

In [69]:
attribute = 'BMI'
problem_type = problem_kind[attribute]
print(problem_type)
# get experiments objects that satisfy all three conditions. Note that running time is in seconds.
experiments = project.get_experiments(tag=attribute)

regression


In [70]:
len(experiments)

36

In [71]:
log_mapping = {'regression': ['ESAOTE_6100/val/mae', 'Philips_iU22/val/mae'],
              'binary': ['ESAOTE_6100/val/accuracy', 'ESAOTE_6100/val/p', 'ESAOTE_6100/val/r',
                        'Philips_iU22/val/p', 'Philips_iU22/val/r', 'Philips_iU22/val/accuracy']}

logs_names = log_mapping[problem_type]

In [72]:
metrics_df = pd.DataFrame(columns=['id', *logs_names])
for experiment in experiments:
    df = experiment.get_numeric_channels_values(*logs_names)  # get logs values
   # print(df)
   # df['tags'] = experiment.get_tags()
   # params = experiment.get_parameters()
    output = df.append(df, ignore_index=True)
    df.insert(loc=0, column='id', value=experiment.id)
    metrics_df = metrics_df.append(df, sort=True)


In [73]:
def compute_f1_esaote(entry):
    return hmean([entry['ESAOTE_6100/val/p'], entry['ESAOTE_6100/val/r']])

def compute_f1_philips(entry):
    return hmean([entry['Philips_iU22/val/p'], entry['Philips_iU22/val/r']])

if problem_type == 'binary':
    metrics_df['ESAOTE_6100/val/f1'] = metrics_df.apply(compute_f1_esaote, axis=1)
    metrics_df['Philips_iU22/val/f1'] = metrics_df.apply(compute_f1_philips, axis=1)
    metrics_df['val_f1_gap'] = metrics_df['ESAOTE_6100/val/f1']  -metrics_df['Philips_iU22/val/f1']
else:
    metrics_df['val_mae_gap'] = metrics_df['ESAOTE_6100/val/mae']  -metrics_df['Philips_iU22/val/mae']

metrics_df.rename(columns={'x': 'epoch'},inplace=True)
metrics_df.head(n=5)

,ESAOTE_6100/val/mae,Philips_iU22/val/mae,id,epoch,val_mae_gap
0,3.153554,7.967573,MUS1-254,1.0,-4.814019
1,3.157040,6.695026,MUS1-254,2.0,-3.537985
2,2.407490,6.987386,MUS1-254,3.0,-4.579896
3,2.250008,6.488988,MUS1-254,4.0,-4.238981
4,2.316710,4.677594,MUS1-254,5.0,-2.360884


In [74]:
# grab the best scoring epoch for each experiment
if problem_type == 'binary':
    best_scores = metrics_df.sort_values(['ESAOTE_6100/val/f1'], ascending=[False]).groupby('id').first()
else:
    best_scores = metrics_df.sort_values(['ESAOTE_6100/val/mae'], ascending=[True]).groupby('id').first()

In [75]:
all_data = project.get_leaderboard(tag=attribute).set_index('id').convert_dtypes()
metrics_df['id']=metrics_df['id'].astype(str)
plot_frame = best_scores.join(all_data)
plot_frame.head(n=10)

,ESAOTE_6100/val/mae,Philips_iU22/val/mae,epoch,val_mae_gap,name,created,finished,owner,notes,running_time,...,parameter_mil_pooling,parameter_n_epochs,parameter_n_params_backend,parameter_n_params_classifier,parameter_n_params_pooling,parameter_prediction_target,parameter_problem_type,parameter_source_train,parameter_use_pseudopatients,parameter_val
id,,,,,,,,,,,,,,,,,,,,,
MUS1-254,2.043918,4.784585,9.0,-2.740667,Untitled,2020-06-28 16:24:15.073000+00:00,2020-06-28 17:10:44.553000+00:00,createrandom,,2789,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-255,2.194778,7.143628,8.0,-4.948850,Untitled,2020-06-28 16:24:15.253000+00:00,2020-06-28 17:10:39.732000+00:00,createrandom,,2784,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-256,2.162746,5.451832,10.0,-3.289086,Untitled,2020-06-28 16:24:15.369000+00:00,2020-06-28 17:09:34.923000+00:00,createrandom,,2719,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-257,2.215012,4.310806,9.0,-2.095794,Untitled,2020-06-28 17:09:50.065000+00:00,2020-06-28 17:54:54.493000+00:00,createrandom,,2704,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-258,2.062725,4.198992,9.0,-2.136267,Untitled,2020-06-28 17:10:55.935000+00:00,2020-06-28 17:57:15.592000+00:00,createrandom,,2779,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-259,2.141685,4.847340,8.0,-2.705655,Untitled,2020-06-28 17:11:00.824000+00:00,2020-06-28 17:57:15.058000+00:00,createrandom,,2774,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-260,2.090652,5.108416,9.0,-3.017765,Untitled,2020-06-28 17:55:10.944000+00:00,2020-06-28 18:42:00.429000+00:00,createrandom,,2809,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-261,2.142902,6.405656,7.0,-4.262754,Untitled,2020-06-28 17:57:31.936000+00:00,2020-06-28 18:44:01.296000+00:00,createrandom,,2789,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"
MUS1-262,2.153104,4.922852,10.0,-2.769748,Untitled,2020-06-28 17:57:32.248000+00:00,2020-06-28 18:43:56.625000+00:00,createrandom,,2784,...,<NA>,10.0,1.1177025E7,<NA>,<NA>,BMI,image,ESAOTE_6100_train,<NA>,"['ESAOTE_6100_val', 'Philips_iU22_val']"


In [76]:
if problem_type == 'binary':
    comp_frame = plot_frame.sort_values('ESAOTE_6100/val/f1', ascending=False)[['epoch','parameter_problem_type', 'parameter_mil_pooling', 'parameter_backend_lr', 'parameter_lr', 'val_f1_gap', 'ESAOTE_6100/val/f1', 'ESAOTE_6100/val/accuracy',  'Philips_iU22/val/f1', 'Philips_iU22/val/accuracy']]
else:
    comp_frame = plot_frame.sort_values('ESAOTE_6100/val/mae')[['parameter_problem_type', 'parameter_mil_pooling', 'parameter_lr', 'val_mae_gap', 'ESAOTE_6100/val/mae', 'Philips_iU22/val/mae']]
    

comp_frame['parameter_mil_pooling'] = comp_frame['parameter_mil_pooling'].fillna('NA')
comp_frame

,parameter_problem_type,parameter_mil_pooling,parameter_lr,val_mae_gap,ESAOTE_6100/val/mae,Philips_iU22/val/mae
id,,,,,,
MUS1-254,image,NA,0.02598707578280313,-2.740667,2.043918,4.784585
MUS1-258,image,NA,0.029107776061145098,-2.136267,2.062725,4.198992
MUS1-265,image,NA,0.04884361542527902,-3.012609,2.077369,5.089978
MUS1-260,image,NA,0.06521887785025218,-3.017765,2.090652,5.108416
MUS1-264,image,NA,0.0359446384673467,-2.598805,2.101757,4.700563
MUS1-259,image,NA,0.09686871270945754,-2.705655,2.141685,4.847340
MUS1-261,image,NA,0.01683277555673535,-4.262754,2.142902,6.405656
MUS1-262,image,NA,0.05442926096619771,-2.769748,2.153104,4.922852
MUS1-256,image,NA,0.07728821731597248,-3.289086,2.162746,5.451832


In [77]:
if problem_type == 'binary':
    print(comp_frame.groupby(['parameter_problem_type','parameter_mil_pooling']).min()['val_f1_gap'])
else:
    print(comp_frame.groupby(['parameter_problem_type','parameter_mil_pooling']).max()['val_mae_gap'])

parameter_problem_type  parameter_mil_pooling
bag                     attention               -0.137249
                        mean                    -0.954675
image                   NA                      -2.095794
Name: val_mae_gap, dtype: float64


In [78]:
import plotly.express as px
plot_frame['parameter_lr'] = plot_frame['parameter_lr'].astype(float)
#plot_frame['parameter_backend_lr'] = plot_frame['parameter_backend_lr'].astype(float)

#plot_frame.drop(columns=['tags'], inplace=True)
fig = px.parallel_coordinates(plot_frame, dimensions=['parameter_lr', 'ESAOTE_6100/val/f1'])
fig.show()



ValueError: Value of 'dimensions_1' is not the name of a column in 'data_frame'. Expected one of ['ESAOTE_6100/val/mae', 'Philips_iU22/val/mae', 'epoch', 'val_mae_gap', 'name', 'created', 'finished', 'owner', 'notes', 'running_time', 'size', 'tags', 'channel_ESAOTE_6100/val/loss', 'channel_ESAOTE_6100/val/mae', 'channel_ESAOTE_6100/val/max_att', 'channel_ESAOTE_6100/val/mean', 'channel_ESAOTE_6100/val/mean_att', 'channel_ESAOTE_6100/val/min_att', 'channel_ESAOTE_6100/val/var', 'channel_ESAOTE_6100/val/var_att', 'channel_ESAOTE_6100/val_image/loss', 'channel_ESAOTE_6100/val_image/mae', 'channel_ESAOTE_6100/val_image/mean', 'channel_ESAOTE_6100/val_image/var', 'channel_Philips_iU22/val/loss', 'channel_Philips_iU22/val/mae', 'channel_Philips_iU22/val/max_att', 'channel_Philips_iU22/val/mean', 'channel_Philips_iU22/val/mean_att', 'channel_Philips_iU22/val/min_att', 'channel_Philips_iU22/val/var', 'channel_Philips_iU22/val/var_att', 'channel_Philips_iU22/val_image/loss', 'channel_Philips_iU